In [1]:
import pandas as pd
import numpy as np
from scipy.stats import *

In [2]:
df = pd.read_csv('Data/all_df_before_1101.csv', index_col = 0)

In [3]:
del df['Recnum']
df['Random'] = np.random.random(df.shape[0])

In [4]:
df.head()

,Amount_avg_card_1,Amount_avg_card_3,Amount_avg_card_7,Amount_avg_card_14,Amount_avg_card_30,Amount_max_card_1,Amount_max_card_3,Amount_max_card_7,Amount_max_card_14,Amount_max_card_30,...,AM1_ACC7,AM1_ACC14,AM1_ACC30,AM1_ANM7,AM1_ANM14,AM1_ANM30,AM1_AAM7,AM1_AAM14,AM1_AAM30,Random
0,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,1.0,1.0,1.0,25.34,50.68,108.6,1.0,1.0,1.0,0.522589
1,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,31.42,...,1.0,1.0,1.0,219.94,439.88,942.6,1.0,1.0,1.0,0.065633
2,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,178.49,...,1.0,1.0,1.0,1249.43,2498.86,5354.7,1.0,1.0,1.0,0.361860
3,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,2.0,2.0,2.0,25.34,50.68,108.6,2.0,2.0,2.0,0.812506
4,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,3.62,...,3.0,3.0,3.0,25.34,50.68,108.6,3.0,3.0,3.0,0.920152


In [5]:
Fraud_1 = df[df['Fraud'] == 1]
Fraud_0 = df[df['Fraud'] == 0]
(Fraud_1.shape, Fraud_0.shape)

((880, 373), (83090, 373))

In [6]:
columns = df.columns.values

In [7]:
df_dict = {}
df_dict['KS'] = {}
for column in columns:
    df_dict['KS'][column] = ks_2samp(Fraud_0[column], Fraud_1[column])[0]

In [8]:
KS_FDR = pd.DataFrame(df_dict)

In [9]:
KS_FDR.sort_values('KS', ascending=False).head()

,KS
Fraud,1.000000
Amount_sum_card_merchant_7,0.677490
Amount_sum_card_merchant_14,0.673334
Amount_sum_card_state_14,0.668354
Amount_sum_card_zip_14,0.668305


In [10]:
int(df.shape[0] * 0.03)

2519

In [11]:
%%time
df_dict['FDR'] = {}
for column in columns:
    top = df.sort_values(column, ascending=False).head(int(df.shape[0] * 0.03))['Fraud'].values
    bottom = df.sort_values(column, ascending=True).head(int(df.shape[0] * 0.03))['Fraud'].values
    FDR_top = sum(top) / Fraud_1.shape[0]
    FDR_bottom = sum(bottom) / Fraud_1.shape[0]
    df_dict['FDR'][column] = max(FDR_top, FDR_bottom)

CPU times: user 1min 45s, sys: 54 s, total: 2min 39s
Wall time: 2min 40s


In [12]:
KS_FDR = pd.DataFrame(df_dict)

In [21]:
KS_FDR["KS_RANK"] = KS_FDR['KS'].rank(ascending = False)
KS_FDR["FDR_RANK"] = KS_FDR['FDR'].rank(ascending = False)
KS_FDR["AVG_RANK"] = (KS_FDR["KS_RANK"] + KS_FDR["FDR_RANK"]) / 2

In [24]:
KS_FDR.sort_values('AVG_RANK', ascending=True).head()

,KS,FDR,KS_RANK,FDR_RANK,AVG_RANK
Fraud,1.000000,1.000000,1.0,1.0,1.0
Amount_sum_card_merchant_7,0.677490,0.635227,2.0,2.0,2.0
Amount_sum_card_merchant_14,0.673334,0.625000,3.0,3.0,3.0
Amount_sum_card_state_7,0.667176,0.619318,6.0,4.0,5.0
Amount_sum_card_zip_7,0.667104,0.617045,7.0,5.0,6.0


In [14]:
KS_FDR.to_csv('Data/KS_FDR.csv')